# AutoEncoder Study

## Colab setting

In [ ]:
# Colab
# from google.colab import drive
# drive.mount("/content/drive")

In [ ]:
# Colab
# import pandas as pd
# train_df = pd.read_csv('./drive/MyDrive/data/train.csv')
# train_df = train_df.drop(columns=['ID'])
# val_df = pd.read_csv('./drive/MyDrive/data/val.csv')
# val_df = val_df.drop(columns=['ID'])
# test_df = pd.read_csv('./drive/MyDrive/data/test.csv')
# test_df = test_df.drop(columns=['ID'])

## Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import f1_score

## Data Load

In [2]:
# Local
train_df = pd.read_csv('./data/train.csv')
train_df = train_df.drop(columns=['ID'])
val_df = pd.read_csv('./data/val.csv')
val_df = val_df.drop(columns=['ID'])
test_df = pd.read_csv('./data/test.csv')
test_df = test_df.drop(columns=['ID'])

In [3]:
# validation data의 정상, 불량 거래 데이터 비율 확인
print('Normals', round(val_df['Class'].value_counts()[0]/len(val_df) * 100,2), '% of the dataset')
print('Frauds', round(val_df['Class'].value_counts()[1]/len(val_df) * 100,2), '% of the dataset')

Normals 99.89 % of the dataset
Frauds 0.11 % of the dataset


## Pytorch

In [ ]:
# device 설정, gpu 있을시 gpu사용
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using Device:", device)

### Hyper parameter

In [ ]:
EPOCHS = 400
LR = 1e-2
BS = 16384
SEED = 123

### Fix Seed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED) # Seed 고정

 ### Make DataSet    

In [ ]:
# eval_ mode를 통해 validation, 즉 평가를 위한 데이터 val.df와 train.df 분리
# val_df의 Class인 정상 거래, 비정상 거래 내용을 labels로, 나머지 feature 값을 df로 저장
# train_df의 값을 df로 저장
class MyDataset(Dataset):
    def __init__(self, df, eval_mode):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
        
    def __len__(self):
        return len(self.df)

### Pytorch Data Load

In [ ]:
# shuffle을 통해 데이터 과적합해결(신경망이 데이터의 순서를 예측하지 못하게 한다)
train_dataset = MyDataset(df=train_df, eval_mode=False)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=2)

val_dataset = MyDataset(df = val_df, eval_mode=True)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=2)

### AutoEncoder 구조(신경망)

In [ ]:
# neural network를 이용해서 AutoEncoder Layer 설정
# BatchNorm1d 정규화 레이어 사용
# LeakyReLU 활성화 함수 사용
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.Encoder = nn.Sequential(
            nn.Linear(30,64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Linear(64,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
        )
        self.Decoder = nn.Sequential(
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Linear(64,30),
        )
        
    def forward(self, x):
        x = self.Encoder(x)
        x = self.Decoder(x)
        return x

### Train

In [ ]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        # Loss Function
        self.criterion = nn.L1Loss().to(self.device)
        
    def fit(self, ):
        self.model.to(self.device)
        best_score = 0
        for epoch in range(EPOCHS):
            self.model.train()
            train_loss = []
            for x in iter(self.train_loader):
                x = x.float().to(self.device)
                # 역전파 단계 전에, optimizer 객체를 사용하여 (모델의 학습 가능한 가중치인) 갱신할
                # 변수들에 대한 모든 변화도(gradient)를 0으로 만듭니다. 이렇게 하는 이유는 기본적으로 
                # .backward()를 호출할 때마다 변화도가 버퍼(buffer)에 (덮어쓰지 않고) 누적되기
                # 때문입니다. 더 자세한 내용은 torch.autograd.backward에 대한 문서를 참조하세요.
                self.optimizer.zero_grad()

                # AutoEncoder 통과한 예측값
                _x = self.model(x)
                # 원래 값과 예측값의 L1loss값 즉 손실함수 적용(오차)
                loss = self.criterion(x, _x)

                # 역전파 단계: 모델의 매개변수들에 대한 손실의 변화도를 계산합니다.
                loss.backward()
                # optimizer의 step 함수를 호출하면 매개변수가 갱신됩니다.
                self.optimizer.step()

                train_loss.append(loss.item())

            score = self.validation(self.model, 0.95)
            print(f'Epoch : [{epoch}] Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')

            if self.scheduler is not None:
                self.scheduler.step(score)

            if best_score < score:
                best_score = score
                torch.save(model.module.state_dict(), './best_model.pth', _use_new_zipfile_serialization=False)
    
    def validation(self, eval_model, thr):
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        #  model.eval()는 이런 layer들의 동작을 inference(eval) mode로 바꿔준다는 목적
        eval_model.eval()
        pred = []
        true = []
        # torch.no_grad()의 주된 목적은 autograd(자동으로 gradient를 트래킹)를 끔으로써 메모리 사용량을 줄이고 연산 속도를 높히기 위함
        with torch.no_grad():
            for x, y in iter(self.val_loader):
                x = x.float().to(self.device)

                _x = self.model(x)
                diff = cos(x, _x).cpu().tolist()
                #유사도 0.95보다 작은것은 이상거래 1, 아닌 것은 정상거래 0
                batch_pred = np.where(np.array(diff)<thr, 1,0).tolist()
                pred += batch_pred
                true += y.tolist()

        return f1_score(true, pred, average='macro')

In [ ]:
model = nn.DataParallel(AutoEncoder())
model.eval()

# optim 패키지를 사용하여 모델의 가중치를 갱신할 optimizer를 정의합니다.
optimizer = torch.optim.Adam(params = model.parameters(), lr = LR)
# 학습률 개선 scheduler, patience번 정체되면 학습률 factor와 곱한다. 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)

trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()

### 추론

In [ ]:
# 학습된 내용 불러오기
model = AutoEncoder()
model.load_state_dict(torch.load('./best_model.pth'))
model = nn.DataParallel(model)
model.eval()

In [ ]:
test_dataset = MyDataset(test_df, False)
test_loader = DataLoader(test_dataset, batch_size=BS, shuffle=False, num_workers=2)

In [ ]:
def prediction(model, thr, test_loader, device):
    model.to(device)
    model.eval()
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    pred = []
    with torch.no_grad():
        for x in iter(test_loader):
            x = x.float().to(device)
            
            _x = model(x)
            
            diff = cos(x, _x).cpu().tolist()
            batch_pred = np.where(np.array(diff)<thr, 1,0).tolist()
            pred += batch_pred
    return pred

In [ ]:
preds = prediction(model, 0.95, test_loader, device)

In [ ]:
submit = pd.read_csv('./drive/MyDrive/data/sample_submission.csv')
submit['Class'] = preds
submit.to_csv('./autoencoder_test_hwan.csv', index=False)